# Chains in Langchain

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [1]:
!pip install -U langchain-community
!pip install langcahin
!pip install langchain-huggingface
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement langcahin (from versions: none)
ERROR: No matching distribution found for langcahin


In [2]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN= userdata.get('HUGGING_FACE_API_KEY')

from huggingface_hub import login
login(token = HUGGINGFACEHUB_API_TOKEN)

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
import transformers, torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token  # Set pad_token to eos_token

model_kwargs = {'temperature':0.3}
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = model_kwargs)

chatModel= ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

In [4]:
from langchain.chains import LLMChain

In [5]:
chain = LLMChain(llm=chatModel, prompt=prompt_template_name ,verbose=True)

<ipython-input-5-05b74f1d5919>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chatModel, prompt=prompt_template_name ,verbose=True)


In [6]:
response=chain.run("indian")
print(response)

<ipython-input-6-6daa3096a481>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain.run("indian")
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food. Suggest a fency name for this.

> Finished chain.
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

I want to open a restaurant for indian food. Suggest a fency name for this.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here are some fantasy name suggestions for your Indian restaurant:

1. **Tandoori Nights**: This name evokes the warm, cozy atmosphere of an Indian tandoor oven and suggests a romantic evening out.
2. **Saffron Spice Co.**: Saffron is a key ingredient in many Indian dishes, and the "Co." suggests a company that's modern and trendy.
3. **Bollywood Bites**: This name combines the popular Indian film genre with the idea of small, shareable bites, making it perfect for a casual dining experience.
4. **Mumbai Morsels**: Mumbai is a majo

# Simple Sequetial chain

In [7]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=chatModel, prompt=prompt_template_name)

In [8]:
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=chatModel, prompt=prompt_template_items)

In [9]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [ food_items_chain,name_chain], verbose=True)

In [10]:
response=chain.run("Indian")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new SimpleSequentialChain chain...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Suggest some menu items for Indian<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here are some popular and delicious menu items from Indian cuisine:

**Appetizers**

1. Samosas (fried or baked pastry filled with spiced potatoes, peas, and onions)
2. Papri Chaat (a spicy street food made with boiled potatoes, chickpeas, and tamarind sauce)
3. Onion Bhajiyas (fried fritters made with onions, spices, and herbs)
4. Raita (a yogurt-based side dish with cucumber, cumin, and coriander)
5. Pakora (fried fritters made with vegetables or meat, typically served with a side of chutney)

**Main Course**

1. Chicken Tikka Masala (marinated chicken cooked in a creamy tomato-based sauce)
2. Palak Paneer (spinach and cheese curry made with paneer, spinach puree, and spices)
3. Chana Masala (chickpea curry made with onio

# Sequential Chain

In [11]:

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=chatModel, prompt=prompt_template_name, output_key="restaurant_name")

In [12]:

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=chatModel, prompt=prompt_template_items, output_key="menu_items")

In [13]:
from langchain.chains import SequentialChain


chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [14]:
print(chain({"cuisine": "indian"}))

<ipython-input-14-33865b5de443>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain({"cuisine": "indian"}))
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarnin

{'cuisine': 'indian', 'restaurant_name': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 10 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI want to open a restaurant for indian food. Suggest a fency name for this.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere are some fantasy name suggestions for your Indian restaurant:\n\n1. **Tandoori Nights**: This name evokes the warm, cozy atmosphere of an Indian tandoor oven and suggests a romantic evening out.\n2. **Saffron Spice Co.**: Saffron is a key ingredient in many Indian dishes, and the "Co." suggests a company that\'s modern and trendy.\n3. **Bollywood Bites**: This name combines the popular Indian film genre with the idea of small, shareable bites, making it perfect for a casual dining experience.\n4. **Mumbai Morsels**: Mumbai is a major city in India, and "Morsels" implies a variety of flavors and textures in each dish.\n5. **Garam